Задача поиск похожих по эмбеддингам Скачиваем датасет

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz Или данные можно скачать тут https://github.com/IlyaGusev/gazeta

from corus import load_ods_gazeta path = 'gazeta.csv.gz' records = load_ods_gazeta(path) next(records) что надо сделать

на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу) Проверить насколько хорошо работают подходы

In [ ]:
!pip install transforms

In [ ]:
!pip install corus

In [ ]:
!pip install tensorflow

In [ ]:
!pip install transformers

In [ ]:
!pip install annoy

In [ ]:
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



from corus import load_ods_gazeta

import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer,  TFBertModel
from transformers import MBartTokenizer, MBartForConditionalGeneration

import annoy
import time

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru",)
model = TFBertModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
vector_size = 1024

In [ ]:
!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

In [ ]:
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)

In [ ]:
article_lenght = 128
article_index = annoy.AnnoyIndex(vector_size,'angular')
index_map = {}
counter = 0

nw = time.time()
for record in load_ods_gazeta(path):
    
    index_map[counter] = record.text
    
    tokens = tokenizer([index_map[counter]], max_length=article_lenght, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='tf')
    vector = model(**tokens)
    

    
    token_embeddings = vector[0][0]
    input_mask = tokens['attention_mask'][0]
    sum_embeddings = token_embeddings[input_mask == 1]
    embed = tf.reduce_sum(sum_embeddings, 0)/tf.reduce_sum(tf.cast(input_mask, dtype=tf.float32))

    
    article_index.add_item(counter, embed)
          
    counter += 1
        
    if (time.time() - nw) // 60 > 30:
        break

article_index.build(10)

In [ ]:
def get_response(model, tokenizer, question, article_index,  answers_nmb=3):
    
    tokens = tokenizer([question], max_length=article_lenght, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='tf')
    vector = model(**tokens)
    
    token_embeddings = vector[0][0]
    input_mask = tokens['attention_mask'][0]
    sum_embeddings = token_embeddings[input_mask == 1]
    embed = tf.reduce_sum(sum_embeddings, 0)/tf.reduce_sum(tf.cast(input_mask, dtype=tf.float32))

    answers = article_index.get_nns_by_vector(embed, answers_nmb)
    return [index_map[i] for i in answers]

In [ ]:
question = "собака друг человека"
get_response(model, tokenizer, question, article_index,  answers_nmb=5)

In [ ]:
question = "Тише едешь дальше будешь"
get_response(model, tokenizer, question, article_index,  answers_nmb=5)

In [ ]:
question = "Спартак чемпион"
get_response(model, tokenizer, question, article_index,  answers_nmb=5)